In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words('english')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

import matplotlib.pyplot as plt
import seaborn as sns

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/isuru/nltk_data...


True

In [33]:
train_df = pd.read_csv('../resources/standard_terms.csv')
test_df = pd.read_csv('../resources/data_dictionary_estbb.csv')
train_df.head()

,TERM,ID,DESCRIPTION,PARENT
0,saliva,UBERON_0001836,A fluid produced in the oral cavity by salivar...,NaN
1,sample type,UBERON_0000463,"Material anatomical entity in a gaseous, liqui...",NaN
2,"Microbiome markers (rRNA, etc)",GECKO_0000040,A genomic assay that profiles the microorganis...,NaN
3,genomics,GECKO_0000032,An assay that investigates the genome or genom...,NaN
4,life stage/time point,UBERON_0000105,A spatiotemporal region encompassing some part...,NaN


In [34]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(text) #removing stopwords
    return text

train_df['DESCRIPTION'] = train_df['DESCRIPTION'].fillna('')
# train_df['FEATURE'] = train_df.apply(lambda x: x['TERM'] + ' ' + x['DESCRIPTION'], axis=1)
train_df['FEATURE'] = train_df['TERM']
train_df['FEATURE'] = train_df['FEATURE'].apply(lambda x: clean_text(x))

test_df['DESCRIPTION'] = test_df['DESCRIPTION'].fillna('')
test_df['TERM'] = test_df.apply(lambda x: x['TERM'].replace('.', ' '), axis=1)
# test_df['FEATURE'] = test_df.apply(lambda x: x['TERM'] + ' ' + x['DESCRIPTION'], axis=1)
test_df['FEATURE'] = test_df['DESCRIPTION']
test_df['FEATURE'] = test_df['FEATURE'].apply(lambda x: clean_text(x))

test_df.head()

,TERM,DESCRIPTION,PARENT,FEATURE
0,Person skood,SKOOD,Person,skood
1,Person gender,Gender,Person,gender
2,Person birthDate,Birthdate,Person,birthdate
3,Person birthYear,Year of birth,Person,year birth
4,Person agreementDate,Agreement date,Person,agreement date


In [35]:
x_train = train_df['FEATURE']
x_test = test_df['FEATURE']
y_train = train_df['TERM']

tfidf_vectorizer = TfidfVectorizer()
tfidf_train_vectors = tfidf_vectorizer.fit_transform(x_train)
tfidf_test_vectors = tfidf_vectorizer.transform(x_test)

classifier = RandomForestClassifier()
classifier.fit(tfidf_train_vectors,y_train)
y_pred = classifier.predict(tfidf_test_vectors)

test_df["CLASS"] = y_pred
test_df.head(20)

,TERM,DESCRIPTION,PARENT,FEATURE,CLASS
0,Person skood,SKOOD,Person,skood,height
1,Person gender,Gender,Person,gender,gender
2,Person birthDate,Birthdate,Person,birthdate,age/birthdate
3,Person birthYear,Year of birth,Person,year birth,height
4,Person agreementDate,Agreement date,Person,agreement date,date and time-related information
5,Person ageToday,Age at present,Person,age present,age/birthdate
6,Person ageAtAgreement,Age at the agreement date,Person,age agreement date,age/birthdate
7,Person ageGroupAtAgreement,,Person,,height
8,Person deathDate,Date of death,Person,date death,cause of death
9,Person deathYear,Year of death,Person,year death,cause of death
